<h1>Pipeline Final</h1>

## Chargement des packages

In [ ]:
# importation de structures de données utiles
import pandas as pd
import numpy as np

# importation de bibliothèques diverses
import os
import gc
import pickle
import time
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

# pour une largeur de cellule à 100 % dans Jupyter Notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# sklearn
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [ ]:
!pip install xgboost

In [ ]:
!pip install lightgbm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !ls /content/drive/MyDrive/Scoring-Projet/data

## Pipeline

In [ ]:

# --- 1. Définition des fonctions de prétraitement et d'ingénierie des caractéristiques ---
# Encapsulation des fonctions de prétraitement et de chargement des fichiers

class PreprocessingHandler:
    def __init__(self, file_directory=''):
        self.file_directory = file_directory
        self.relational_table = None
        # Changé en final_cols_for pour le distinguer de columns_for_modelling
        self.final_cols = None
        self.columns_for_modelling = None
        self.xgbr_ext_1 = None
        self.xgbr_ext_2 = None
        self.xgbr_ext_3 = None
        self.cnt_payment_lgbmr = None
        self.TARGET = None
        self.knn_target_500_neighbors = None
        self.group_interactions_tables = []
        self.load_required_files()

    def load_required_files(self):
        '''
        Fonction pour charger tous les fichiers nécessaires au prétraitement et à la prédiction.
        Elle est appelée lors de l'initialisation de la classe.

        Entrées :
            self

        Sorties :
            Aucune
        '''
        print("Chargement des fichiers requis pour le prétraitement...")
        try:
            # Chargement des fichiers importants
            with open(self.file_directory + 'relational_table1.pkl', 'rb') as f:
                self.relational_table = pickle.load(f)
            # Chargement de la liste finale des colonnes pour l’alignement
            # Changé en final_cols_for_align
            with open(self.file_directory + 'columns_to_select.pkl', 'rb') as f:
                self.final_cols = pickle.load(f)
            # Chargement des colonnes utilisées pour la modélisation des EXT_SOURCE
            with open(self.file_directory + 'columns_for_ext_values_predictor.pkl', 'rb') as f:
                self.columns_for_modelling = pickle.load(f)
            # Chargement des modèles XGBRegressor pour prédire les champs EXT_SOURCE manquants
            with open(self.file_directory + 'nan_EXT_SOURCE_1_xgbr_model.pkl', 'rb') as f:
                self.xgbr_ext_1 = pickle.load(f)
            # Nom de fichier corrigé pour inclure l'extension .pkl
            with open(self.file_directory + 'nan_EXT_SOURCE_2_xgbr_model.pkl', 'rb') as f:
                self.xgbr_ext_2 = pickle.load(f)
            with open(self.file_directory + 'nan_EXT_SOURCE_3_xgbr_model.pkl', 'rb') as f:
                self.xgbr_ext_3 = pickle.load(f)
            # Chargement du modèle LGBMRegressor pour prédire le nombre de paiements
            with open(self.file_directory + 'cnt_payment_predictor_lgbmr.pkl', 'rb') as f:
                self.cnt_payment_lgbmr = pickle.load(f)
            # Chargement des valeurs cibles d'entraînement
            with open(self.file_directory + 'Train_TARGET.pkl', 'rb') as f:
                self.TARGET = pickle.load(f)
            # Chargement du modèle KNN pour la caractéristique TARGET_500_neighbors
            with open(self.file_directory + 'KNN_model_TARGET_500_neighbors.pkl', 'rb') as f:
                self.knn_target_500_neighbors = pickle.load(f)
            # Chargement des interactions groupées pour plusieurs regroupements
            file_names = ['Application_train_grouped_interactions_NAME_CONTRACT_TYPE_NAME_INCOME_TYPE_OCCUPATION_TYPE.pkl',
                          'Application_train_grouped_interactions_CODE_GENDER_NAME_FAMILY_STATUS_NAME_INCOME_TYPE.pkl',
                          'Application_train_grouped_interactions_FLAG_OWN_CAR_FLAG_OWN_REALTY_NAME_INCOME_TYPE.pkl',
                          'Application_train_grouped_interactions_NAME_EDUCATION_TYPE_NAME_INCOME_TYPE_OCCUPATION_TYPE.pkl',
                          'Application_train_grouped_interactions_OCCUPATION_TYPE_ORGANIZATION_TYPE.pkl',
                          'Application_train_grouped_interactions_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY.pkl']
            for group_interactions_file_name in file_names:
                group_file_path = self.file_directory + 'Grouped Interactions/' + group_interactions_file_name
                if os.path.exists(group_file_path):
                    with open(group_file_path, 'rb') as f:
                        self.group_interactions_tables.append(pickle.load(f))
                else:
                    print(f"Attention : fichier d'interactions groupées introuvable à {group_file_path}. Ignoré.")

            print("Tous les fichiers requis ont été chargés avec succès.")
        except FileNotFoundError as e:
            print(f"Erreur de chargement du fichier : {e}. Veuillez vérifier que tous les fichiers .pkl nécessaires sont présents dans le répertoire spécifié et ses sous-dossiers.")
            raise
        except Exception as e:
            print(f"Une erreur inattendue est survenue lors du chargement des fichiers : {e}")
            raise

    def create_new_features(self, data):
        '''
        Fonction pour créer de nouvelles variables après la fusion des caractéristiques,
        en utilisant les interactions entre différentes tables.

        Entrées :
            data : DataFrame

        Sorties :
            DataFrame avec de nouvelles variables
        '''
        print("Creating new features...")
        # Liste des colonnes de "flag" à supprimer car elles sont considérées comme non utiles
        flag_cols_to_drop = ['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_10','FLAG_DOCUMENT_12',
                             'FLAG_DOCUMENT_20']
        data = data.drop(flag_cols_to_drop, axis = 1, errors='ignore') # Suppression des colonnes spécifiées
        # Conversion de l'âge en années (DAYS_BIRTH est généralement en jours négatifs)
        data['DAYS_BIRTH'] = data['DAYS_BIRTH'] * -1 / 365
        # Suppression des points erronés ou des valeurs aberrantes
        # Les valeurs 365243 dans DAYS_EMPLOYED sont remplacées par NaN (valeur manquante)
        data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].replace({365243: np.nan})
        # Les valeurs aberrantes (> 30) dans les colonnes de cercle social sont remplacées par NaN
        data['OBS_30_CNT_SOCIAL_CIRCLE'] = data['OBS_30_CNT_SOCIAL_CIRCLE'].replace(to_replace=r'^3[1-9]+|[4-9][0-9]*$', value=np.nan, regex=True)
        data['OBS_60_CNT_SOCIAL_CIRCLE'] = data['OBS_60_CNT_SOCIAL_CIRCLE'].replace(to_replace=r'^3[1-9]+|[4-9][0-9]*$', value=np.nan, regex=True)

        # Remplissage des valeurs NaN pour les colonnes catégorielles avec 'XNA'
        categorical_columns = data.dtypes[data.dtypes == 'object'].index.tolist() # Obtient la liste des colonnes de type 'object' (catégorielles)
        data[categorical_columns] = data[categorical_columns].fillna('XNA') # Remplit les NaN avec 'XNA'
        # Conversion des colonnes de notation de région en type 'object' (catégoriel)
        data['REGION_RATING_CLIENT'] = data['REGION_RATING_CLIENT'].astype('object')
        data['REGION_RATING_CLIENT_W_CITY'] = data['REGION_RATING_CLIENT_W_CITY'].astype('object')
        # Comptage du nombre total de valeurs manquantes pour chaque demande (ligne)
        data['MISSING_VALS_TOTAL_APP'] = data.isna().sum(axis = 1) # Calcule la somme des NaN par ligne

        # Nous devons prédire les valeurs manquantes de EXT_SOURCE s'il y en a
        columns_for_modelling_ext = list(self.columns_for_modelling)
        xgbr_ext_models = [self.xgbr_ext_2, self.xgbr_ext_3, self.xgbr_ext_1]
        # Itération sur les colonnes EXT_SOURCE pour prédire les valeurs manquantes
        for index, ext_col in enumerate(['EXT_SOURCE_2','EXT_SOURCE_3','EXT_SOURCE_1']):
            # Vérifie si des valeurs sont manquantes dans la colonne EXT_SOURCE actuelle
            if ext_col in data.columns and data[ext_col].isna().sum() > 0:
                # Sélectionne les lignes avec des valeurs manquantes et les colonnes pertinentes pour la prédiction
                cols_present_in_data = [col for col in columns_for_modelling_ext if col in data.columns]
                X_test_missing = data.loc[data[ext_col].isna(), cols_present_in_data]
                try:
                    predicted_values = xgbr_ext_models[index].predict(X_test_missing)
                    data.loc[data[ext_col].isna(), ext_col] = predicted_values
                except Exception as e:
                     print(f"Error predicting missing values for {ext_col}: {e}")
                     data.loc[data[ext_col].isna(), ext_col] = data[ext_col].median() if not data[ext_col].median() == np.nan else 0.0 # Or 0.0 or mean()

            # Ajoute la colonne prédite aux colonnes utilisées pour la modélisation pour la prédiction des colonnes suivantes
            if ext_col not in columns_for_modelling_ext:
                 columns_for_modelling_ext.append(ext_col)

        # Création de nouvelles caractéristiques numériques

        # Caractéristiques de revenu et de crédit
        data['CREDIT_INCOME_RATIO'] = data['AMT_CREDIT'] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Ratio crédit / revenu
        data['CREDIT_ANNUITY_RATIO'] = data['AMT_CREDIT'] / (data['AMT_ANNUITY'] + 0.00001) # Ratio crédit / annuité
        data['ANNUITY_INCOME_RATIO'] = data['AMT_ANNUITY'] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Ratio annuité / revenu
        data['INCOME_ANNUITY_DIFF'] = data['AMT_INCOME_TOTAL'] - data['AMT_ANNUITY'] # Différence revenu - annuité
        data['CREDIT_GOODS_RATIO'] = data['AMT_CREDIT'] / (data['AMT_GOODS_PRICE'] + 0.00001) # Ratio crédit / prix des biens
        data['CREDIT_GOODS_DIFF'] = data['AMT_CREDIT'] - data['AMT_GOODS_PRICE'] + 0.00001 # Différence crédit - prix des biens
        data['GOODS_INCOME_RATIO'] = data['AMT_GOODS_PRICE'] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Ratio prix des biens / revenu
        # Add checks for EXT_SOURCE_3 before division
        data['INCOME_EXT_RATIO'] = data['AMT_INCOME_TOTAL'] / (data['EXT_SOURCE_3'].replace(0, np.nan) + 0.00001) # Ratio revenu / EXT_SOURCE_3
        data['CREDIT_EXT_RATIO'] = data['AMT_CREDIT'] / (data['EXT_SOURCE_3'].replace(0, np.nan) + 0.00001) # Ratio crédit / EXT_SOURCE_3
        # Ratios et différences d'âge
        data['AGE_EMPLOYED_DIFF'] = data['DAYS_BIRTH'] - data['DAYS_EMPLOYED'] # Différence âge - jours travaillés
        data['EMPLOYED_TO_AGE_RATIO'] = data['DAYS_EMPLOYED'] / (data['DAYS_BIRTH'] + 0.00001) # Ratio jours travaillés / âge
        # Ratios liés à la voiture
        data['CAR_EMPLOYED_DIFF'] = data['OWN_CAR_AGE'] - data['DAYS_EMPLOYED'] # Différence âge voiture - jours travaillés
        data['CAR_EMPLOYED_RATIO'] = data['OWN_CAR_AGE'] / (data['DAYS_EMPLOYED']+0.00001) # Ratio âge voiture / jours travaillés
        data['CAR_AGE_DIFF'] = data['DAYS_BIRTH'] - data['OWN_CAR_AGE'] # Différence âge - âge voiture
        data['CAR_AGE_RATIO'] = data['OWN_CAR_AGE'] / (data['DAYS_BIRTH'] + 0.00001) # Ratio âge voiture / âge
        # Somme des drapeaux de contact
        data['FLAG_CONTACTS_SUM'] = data['FLAG_MOBIL'] + data['FLAG_EMP_PHONE'] + data['FLAG_WORK_PHONE'] + data[
                                     'FLAG_CONT_MOBILE'] + data['FLAG_PHONE'] + data['FLAG_EMAIL'] # Somme des indicateurs de contact

        data['HOUR_PROCESS_CREDIT_MUL'] = data['AMT_CREDIT'] * data['HOUR_APPR_PROCESS_START'] # Produit crédit * heure de début de processus
        # Membres de la famille
        data['CNT_NON_CHILDREN'] = data['CNT_FAM_MEMBERS'] - data['CNT_CHILDREN'] # Nombre de membres de la famille sans enfants
        data['CHILDREN_INCOME_RATIO'] = data['CNT_CHILDREN'] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Ratio enfants / revenu total
        data['PER_CAPITA_INCOME'] = data['AMT_INCOME_TOTAL'] / (data['CNT_FAM_MEMBERS'] + 1) # Revenu par membre de la famille
        # Notations de région
        # Ensure columns are numeric before calculation
        data['REGIONS_RATING_INCOME_MUL'] = (data['REGION_RATING_CLIENT'].astype(float) + data['REGION_RATING_CLIENT_W_CITY'].astype(float)) * data['AMT_INCOME_TOTAL'] / 2 # Produit moyen des notations de région par le revenu
        data['REGION_RATING_MAX'] = [max(float(ele1), float(ele2)) for ele1, ele2 in zip(data['REGION_RATING_CLIENT'], data['REGION_RATING_CLIENT_W_CITY'])] # Maximum des notations de région
        # The next line was a copy-paste error in original code, it should be MIN not MAX again. Corrected to MIN
        data['REGION_RATING_MIN'] = [min(float(ele1), float(ele2)) for ele1, ele2 in zip(data['REGION_RATING_CLIENT'], data['REGION_RATING_CLIENT_W_CITY'])] # Minimum des notations de région
        data['REGION_RATING_MEAN'] = (data['REGION_RATING_CLIENT'].astype(float) + data['REGION_RATING_CLIENT_W_CITY'].astype(float)) / 2 # Moyenne des notations de région
        data['REGION_RATING_MUL'] = data['REGION_RATING_CLIENT'].astype(float) * data['REGION_RATING_CLIENT_W_CITY'].astype(float) # Produit des notations de région
        # Drapeaux de région combinés
        data['FLAG_REGIONS'] = data['REG_REGION_NOT_LIVE_REGION'] + data['REG_REGION_NOT_WORK_REGION'] + data['LIVE_REGION_NOT_WORK_REGION']+data[
                                 'REG_CITY_NOT_LIVE_CITY'] + data['REG_CITY_NOT_WORK_CITY'] + data['LIVE_CITY_NOT_WORK_CITY'] # Somme des drapeaux de non-correspondance des régions
        # Sources externes (EXT_SOURCE)
        # Add checks for EXT_SOURCE columns before calculation
        ext_cols = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
        for col in ext_cols:
            if col not in data.columns:
                data[col] = np.nan # Ensure columns exist, fill with NaN if missing
        data['EXT_SOURCE_MEAN'] = data[ext_cols].mean(axis=1) # Moyenne des sources externes
        data['EXT_SOURCE_MUL'] = data['EXT_SOURCE_1'].fillna(data['EXT_SOURCE_1'].mean()) * data['EXT_SOURCE_2'].fillna(data['EXT_SOURCE_2'].mean()) * data['EXT_SOURCE_3'].fillna(data['EXT_SOURCE_3'].mean()) # Produit des sources externes
        data['EXT_SOURCE_MAX'] = data[ext_cols].max(axis=1) # Maximum des sources externes
        data['EXT_SOURCE_MIN'] = data[ext_cols].min(axis=1) # Minimum des sources externes
        data['EXT_SOURCE_VAR'] = data[ext_cols].var(axis=1) # Variance des sources externes
        data['WEIGHTED_EXT_SOURCE'] = data['EXT_SOURCE_1'].fillna(0) * 2 + data['EXT_SOURCE_2'].fillna(0) * 3 + data['EXT_SOURCE_3'].fillna(0) * 4 # Somme pondérée des sources externes
        # Scores d'appartement
        apartment_avg_cols = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',
                              'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG',
                              'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
                              'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG']
        for col in apartment_avg_cols:
            if col not in data.columns:
                data[col] = 0.0
        data['APARTMENTS_SUM_AVG'] = data[apartment_avg_cols].sum(axis=1)

        apartment_mode_cols = ['APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
                               'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE',
                               'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE',
                               'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'TOTALAREA_MODE']
        for col in apartment_mode_cols:
            if col not in data.columns:
                data[col] = 0.0
        data['APARTMENTS_SUM_MODE'] = data[apartment_mode_cols].sum(axis=1)

        apartment_medi_cols = ['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
                               'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI',
                               'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
                               'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI']
        for col in apartment_medi_cols:
            if col not in data.columns:
                data[col] = 0.0
        data['APARTMENTS_SUM_MEDI'] = data[apartment_medi_cols].sum(axis=1)

        data['INCOME_APARTMENT_AVG_MUL'] = data['APARTMENTS_SUM_AVG'] * data['AMT_INCOME_TOTAL'] # Produit des scores moyens d'appartement par le revenu
        data['INCOME_APARTMENT_MODE_MUL'] = data['APARTMENTS_SUM_MODE'] * data['AMT_INCOME_TOTAL'] # Produit des scores modaux d'appartement par le revenu
        data['INCOME_APARTMENT_MEDI_MUL'] = data['APARTMENTS_SUM_MEDI'] * data['AMT_INCOME_TOTAL'] # Produit des scores médians d'appartement par le revenu
        # OBS et DEF (observations et défauts)
        obs_def_cols = ['OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
        for col in obs_def_cols:
             if col not in data.columns:
                 data[col] = np.nan

        data['OBS_30_60_SUM'] = data['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(0) + data['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0) # Somme des observations à 30 et 60 jours
        data['DEF_30_60_SUM'] = data['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0) + data['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0) # Somme des défauts à 30 et 60 jours
        data['OBS_DEF_30_MUL'] = data['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(0) * data['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0) # Produit des observations et défauts à 30 jours
        data['OBS_DEF_60_MUL'] = data['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0) * data['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0) # Produit des observations et défauts à 60 jours
        data['SUM_OBS_DEF_ALL'] = data['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(0) + data['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0) + data[
                                     'OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0) + data['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0) # Somme de toutes les observations et défauts
        data['OBS_30_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['OBS_30_CNT_SOCIAL_CIRCLE'].replace(0, np.nan).fillna(data['OBS_30_CNT_SOCIAL_CIRCLE'].median()) + 0.00001) # Ratio crédit / observations à 30 jours
        data['OBS_60_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['OBS_60_CNT_SOCIAL_CIRCLE'].replace(0, np.nan).fillna(data['OBS_60_CNT_SOCIAL_CIRCLE'].median()) + 0.00001) # Ratio crédit / observations à 60 jours
        data['DEF_30_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['DEF_30_CNT_SOCIAL_CIRCLE'].replace(0, np.nan).fillna(data['DEF_30_CNT_SOCIAL_CIRCLE'].median()) + 0.00001) # Ratio crédit / défauts à 30 jours
        data['DEF_60_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['DEF_60_CNT_SOCIAL_CIRCLE'].replace(0, np.nan).fillna(data['DEF_60_CNT_SOCIAL_CIRCLE'].median()) + 0.00001) # Ratio crédit / défauts à 60 jours
        # Somme des drapeaux de documents combinés
        document_flags = ['FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7',
                          'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_13',
                          'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
                          'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_21']
        for col in document_flags:
            if col not in data.columns:
                data[col] = 0
        data['SUM_FLAGS_DOCUMENTS'] = data[document_flags].sum(axis=1) # Somme de tous les drapeaux de documents (sauf ceux supprimés initialement)
        # Changement de détails
        data['DAYS_DETAILS_CHANGE_MUL'] = data['DAYS_LAST_PHONE_CHANGE'] * data['DAYS_REGISTRATION'] * data['DAYS_ID_PUBLISH'] # Produit des jours de changement de téléphone, d'enregistrement et de publication d'ID
        data['DAYS_DETAILS_CHANGE_SUM'] = data['DAYS_LAST_PHONE_CHANGE'] + data['DAYS_REGISTRATION'] + data['DAYS_ID_PUBLISH'] # Somme des jours de changement de téléphone, d'enregistrement et de publication d'ID
        # Enquêtes de crédit
        enq_credit_cols = ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
                           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']
        for col in enq_credit_cols:
            if col not in data.columns:
                data[col] = 0.0
        data['AMT_ENQ_SUM'] = data[enq_credit_cols].sum(axis=1) # Somme des demandes de crédit au bureau
        data['ENQ_CREDIT_RATIO'] = data['AMT_ENQ_SUM'] / (data['AMT_CREDIT'] + 0.00001) # Ratio demandes de crédit / montant du crédit

        # Nous devons prédire le nombre de paiements pour nos données
        test_data_for_cnt_payment = data[['AMT_CREDIT','AMT_ANNUITY']].fillna(0) # Crée un DataFrame temporaire avec les colonnes crédit et annuité
        test_data_for_cnt_payment['CREDIT_ANNUITY_RATIO'] = test_data_for_cnt_payment['AMT_CREDIT'] / (test_data_for_cnt_payment['AMT_ANNUITY'] + 1) # Calcule le ratio crédit / annuité
        cnt_payment = self.cnt_payment_lgbmr.predict(test_data_for_cnt_payment) # Prédit le nombre de paiements en utilisant un modèle LightGBM (stocké dans 'self')
        del test_data_for_cnt_payment # Supprime le DataFrame temporaire pour libérer de la mémoire
        data['EXPECTED_CNT_PAYMENT'] = cnt_payment # Ajoute le nombre de paiements prédit
        data['EXPECTED_INTEREST'] = data['AMT_ANNUITY'] * data['EXPECTED_CNT_PAYMENT'] - data['AMT_CREDIT'] # Calcule l'intérêt attendu
        data['EXPECTED_INTEREST_SHARE'] = data['EXPECTED_INTEREST'] / (data['AMT_CREDIT'] + 0.00001) # Calcule la part d'intérêt attendue
        data['EXPECTED_INTEREST_RATE'] = 2 * 12 * data['EXPECTED_INTEREST'] / (data['AMT_CREDIT'] * (data['EXPECTED_CNT_PAYMENT'] + 1)) # Calcule le taux d'intérêt attendu

        # Prédiction de la moyenne de la variable CIBLE des 500 voisins les plus proches des données de test
        ext_knn_cols = ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','CREDIT_ANNUITY_RATIO']
        for col in ext_knn_cols:
            if col not in data.columns:
                data[col] = np.nan
        test_data_for_neighbors = data[ext_knn_cols].fillna(0) # Prépare les données pour la recherche des voisins
        # Trouve les 500 voisins les plus proches et récupère leurs index
        test_500_neighbors = self.knn_target_500_neighbors.kneighbors(test_data_for_neighbors)[1]
        # Calcule la moyenne de la variable CIBLE pour ces 500 voisins
        data['TARGET_NEIGHBORS_500_MEAN'] = [self.TARGET.iloc[ele].mean() for ele in test_500_neighbors]

        # Création de caractéristiques basées sur les interactions catégorielles
        columns_to_aggregate_on = [
            ['NAME_CONTRACT_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE'],
            ['CODE_GENDER', 'NAME_FAMILY_STATUS', 'NAME_INCOME_TYPE'],
            ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE'],
            ['NAME_EDUCATION_TYPE','NAME_INCOME_TYPE','OCCUPATION_TYPE'],
            ['OCCUPATION_TYPE','ORGANIZATION_TYPE'],
            ['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY']
        ]
        # Pour chaque groupe de colonnes, joindre les statistiques groupées pré-calculées
        for index, group in enumerate(columns_to_aggregate_on):
            group_stats = self.group_interactions_tables[index]  # Récupérer les statistiques groupées (doivent être pré-calculées et stockées dans 'self')

            # Vérifier si toutes les colonnes du groupe existent dans 'data' avant de faire la jointure
            if all(col in data.columns for col in group):
                # Ajouter un suffixe aux colonnes fusionnées pour éviter les conflits si des noms de colonnes se chevauchent
                # En supposant que les colonnes de group_stats sont comme ['group_col1', 'group_col2', 'aggregated_feature']
                # Il faut s'assurer que le nom de la colonne de la caractéristique agrégée soit unique
                # On suppose ici que cette colonne agrégée est la dernière après les colonnes de regroupement dans group_stats
                aggregated_col_name_suffix = '_group_' + '_'.join(group)  # Créer un suffixe unique

                # Renommer les colonnes dans group_stats avant la fusion
                group_stats_renamed = group_stats.copy()
                # En supposant que les colonnes de jointure sont les premières len(group) colonnes
                cols_to_rename = group_stats_renamed.columns[len(group):]
                rename_dict = {col: col + aggregated_col_name_suffix for col in cols_to_rename}
                group_stats_renamed = group_stats_renamed.rename(columns=rename_dict)

                data = data.merge(group_stats_renamed, on=group, how='left')  # Joindre les statistiques au DataFrame principal
            else:
                print(f"Attention : colonnes manquantes dans les données pour la jointure du groupe : {group}. Fusion ignorée.")

        # Codage de réponse pour les colonnes catégorielles
        # Obtient à nouveau la liste des colonnes catégorielles
        categorical_columns = data.dtypes[data.dtypes == 'object'].index.tolist()
        for col in categorical_columns:
            try:
                # Charge le dictionnaire de codage de réponse pour la colonne actuelle
                mapping_file_path = self.file_directory + 'Response Coding/' + f'Response_coding_dict_{col}.pkl'
                if os.path.exists(mapping_file_path):
                    with open(mapping_file_path, 'rb') as f:
                        mapping_dictionary_column = pickle.load(f)
                    data[col + '_0'] = data[col].map(mapping_dictionary_column[0])
                    data[col + '_1'] = data[col].map(mapping_dictionary_column[1])
                    # Supprimer la variable catégorielle d'origine
                    _ = data.pop(col)
                else:
                    print(f"Attention : le dictionnaire de codage de réponse pour la colonne '{col}' est introuvable à l'emplacement {mapping_file_path}. Codage de réponse ignoré pour cette colonne.")
            except Exception as e:
                print(f"Erreur lors du codage de réponse pour la colonne '{col}' : {e}. Codage ignoré.")

        # Fusion avec la table relationnelle
        if 'SK_ID_CURR' in data.columns and self.relational_table is not None:
            if 'SK_ID_CURR' in self.relational_table.columns:
                data = data.merge(self.relational_table, on='SK_ID_CURR', how='left')
            else:
                print("Attention : 'SK_ID_CURR' est absent de relational_table. Fusion ignorée.")
        else:
            print("Attention : 'SK_ID_CURR' est absent de data ou relational_table n'est pas chargée. Fusion ignorée.")

        # Ajout de caractéristiques basées sur les interactions entre différentes tables

        # previous_application columns
        prev_annuity_columns = ['AMT_ANNUITY_MEAN_LAST_5', 'AMT_ANNUITY_SUM_LAST_5', 'AMT_ANNUITY_MAX_LAST_5', 'AMT_ANNUITY_MEAN_FIRST_2',
                                'AMT_ANNUITY_SUM_FIRST_2', 'AMT_ANNUITY_MAX_FIRST_2', 'AMT_ANNUITY_MEAN_ALL', 'AMT_ANNUITY_SUM_ALL', 'AMT_ANNUITY_MAX_ALL']
        for col in prev_annuity_columns:
            if col in data.columns and 'AMT_INCOME_TOTAL' in data.columns:
                data['PREV_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Crée des ratios d'annuité par rapport au revenu

        prev_goods_columns = ['AMT_GOODS_PRICE_MEAN_LAST_5', 'AMT_GOODS_PRICE_MAX_LAST_5', 'AMT_GOODS_PRICE_SUM_LAST_5', 'AMT_GOODS_PRICE_MEAN_FIRST_2',
                              'AMT_GOODS_PRICE_MAX_FIRST_2', 'AMT_GOODS_PRICE_SUM_FIRST_2', 'AMT_GOODS_PRICE_MEAN_ALL', 'AMT_GOODS_PRICE_MAX_ALL',
                              'AMT_GOODS_PRICE_SUM_ALL']
        for col in prev_goods_columns:
            if col in data.columns and 'AMT_INCOME_TOTAL' in data.columns:
                data['PREV_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Crée des ratios de prix des biens par rapport au revenu

        # credit_card_balance columns
        cc_amt_principal_cols = ['AMT_RECEIVABLE_PRINCIPAL_SUM', 'AMT_RECEIVABLE_PRINCIPAL_MEAN', 'AMT_RECEIVABLE_PRINCIPAL_MAX', 'EXP_AMT_RECEIVABLE_PRINCIPAL_LAST']
        for col in cc_amt_principal_cols:
            if col in data.columns and 'AMT_INCOME_TOTAL' in data.columns:
                data['CC_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Crée des ratios de principal à recevoir par rapport au revenu

        cc_amt_recivable_cols = ['AMT_RECIVABLE_SUM', 'AMT_RECIVABLE_MEAN', 'AMT_RECIVABLE_MAX', 'EXP_AMT_RECIVABLE_LAST']
        for col in cc_amt_recivable_cols:
            if col in data.columns and 'AMT_INCOME_TOTAL' in data.columns:
                data['CC_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Crée des ratios d'encours à recevoir par rapport au revenu

        cc_amt_total_receivable_cols = ['AMT_TOTAL_RECEIVABLE_SUM', 'AMT_TOTAL_RECEIVABLE_MEAN', 'AMT_TOTAL_RECEIVABLE_MAX', 'EXP_AMT_TOTAL_RECEIVABLE_LAST']
        for col in cc_amt_total_receivable_cols:
            if col in data.columns and 'AMT_INCOME_TOTAL' in data.columns:
                data['CC_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Crée des ratios du total à recevoir par rapport au revenu

        # installments_payments columns
        installments_payment_cols = ['AMT_PAYMENT_MEAN_MEAN', 'AMT_PAYMENT_MEAN_SUM', 'AMT_PAYMENT_MEAN_MAX', 'AMT_PAYMENT_SUM_MEAN', 'AMT_PAYMENT_SUM_SUM',
                                     'AMT_PAYMENT_SUM_MAX', 'AMT_PAYMENT_MAX_MEAN', 'AMT_PAYMENT_MEAN_LAST_1_YEAR', 'AMT_PAYMENT_SUM_LAST_1_YEAR',
                                     'AMT_PAYMENT_MAX_LAST_1_YEAR', 'AMT_PAYMENT_MEAN_FIRST_5_INSTALLMENTS', 'AMT_PAYMENT_SUM_FIRST_5_INSTALLMENTS',
                                     'AMT_PAYMENT_MAX_FIRST_5_INSTALLMENTS']
        for col in installments_payment_cols:
            if col in data.columns and 'AMT_INCOME_TOTAL' in data.columns:
                data['INSTALLMENTS_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Crée des ratios de paiement par rapport au revenu

        installments_max_installment = ['AMT_INSTALMENT_MEAN_MAX', 'AMT_INSTALMENT_SUM_MAX']
        for col in installments_max_installment:
            if col in data.columns and 'AMT_ANNUITY' in data.columns:
                data['INSTALLMENTS_ANNUITY_' + col + '_RATIO'] = data['AMT_ANNUITY'] / (data[col] + 0.00001) # Crée des ratios d'annuité par rapport aux montants des versements maximaux

        # bureau and bureau_balance columns
        bureau_days_credit_cols = ['DAYS_CREDIT_MEAN_OVERALL', 'DAYS_CREDIT_MEAN_CREDITACTIVE_CLOSED', 'DAYS_CREDIT_MIN_CREDITACTIVE_CLOSED',
                                   'DAYS_CREDIT_MAX_CREDITACTIVE_CLOSED', 'DAYS_CREDIT_LAST_CREDITACTIVE_CLOSED', 'DAYS_CREDIT_MEAN_CREDITACTIVE_ACTIVE',
                                   'DAYS_CREDIT_MIN_CREDITACTIVE_ACTIVE', 'DAYS_CREDIT_MAX_CREDITACTIVE_ACTIVE', 'DAYS_CREDIT_LAST_CREDITACTIVE_ACTIVE',
                                   'DAYS_CREDIT_MEANCREDIT_ACTIVE_REST', 'DAYS_CREDIT_MINCREDIT_ACTIVE_REST', 'DAYS_CREDIT_MAXCREDIT_ACTIVE_REST',
                                   'DAYS_CREDIT_LASTCREDIT_ACTIVE_REST']
        for col in bureau_days_credit_cols:
            if col in data.columns and 'DAYS_EMPLOYED' in data.columns:
                data['BUREAU_' + col + '_EMPLOYED_DIFF'] = data[col] - data['DAYS_EMPLOYED'] # Différence entre jours de crédit et jours travaillés
            if col in data.columns and 'DAYS_REGISTRATION' in data.columns:
                data['BUREAU_' + col + '_REGISTRATION_DIFF'] = data[col] - data['DAYS_REGISTRATION'] # Différence entre jours de crédit et jours d'enregistrement

        bureau_overdue_cols = ['AMT_CREDIT_MAX_OVERDUE_MEAN_OVERALL', 'AMT_CREDIT_SUM_OVERDUE_MEAN_OVERALL', 'AMT_CREDIT_MAX_OVERDUE_MAX_CREDITACTIVE_CLOSED',
                               'AMT_CREDIT_MAX_OVERDUE_SUM_CREDITACTIVE_CLOSED', 'AMT_CREDIT_SUM_OVERDUE_MAX_CREDITACTIVE_CLOSED', 'AMT_CREDIT_SUM_OVERDUE_SUM_CREDITACTIVE_CLOSED',
                               'AMT_CREDIT_MAX_OVERDUE_MAX_CREDITACTIVE_ACTIVE', 'AMT_CREDIT_MAX_OVERDUE_SUM_CREDITACTIVE_ACTIVE', 'AMT_CREDIT_SUM_OVERDUE_MAX_CREDITACTIVE_ACTIVE',
                               'AMT_CREDIT_SUM_OVERDUE_SUM_CREDITACTIVE_ACTIVE', 'AMT_CREDIT_MAX_OVERDUE_MAXCREDIT_ACTIVE_REST', 'AMT_CREDIT_MAX_OVERDUE_SUMCREDIT_ACTIVE_REST',
                               'AMT_CREDIT_SUM_OVERDUE_MAXCREDIT_ACTIVE_REST', 'AMT_CREDIT_SUM_OVERDUE_SUMCREDIT_ACTIVE_REST']
        for col in bureau_overdue_cols:
            if col in data.columns and 'AMT_INCOME_TOTAL' in data.columns:
                data['BUREAU_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Crée des ratios d'arriérés de crédit par rapport au revenu

        bureau_amt_annuity_cols = ['AMT_ANNUITY_MEAN_OVERALL']
        for col in bureau_amt_annuity_cols:
            if col in data.columns and 'AMT_INCOME_TOTAL' in data.columns:
                data['BUREAU_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001) # Crée des ratios d'annuité du bureau par rapport au revenu


        print("Création de nouvelles variables.")
        return data

# --- 2. Classe principale du pipeline (inspirée de testing_class) ---
class FinalPipeline:
    def __init__(self, model_path, file_directory=''):
        self.model_path = model_path
        self.file_directory = file_directory
        self.model = None
        self.threshold = 0.314
        self.preprocessing_handler = PreprocessingHandler(file_directory)  # Initialiser le gestionnaire de prétraitement
        self.load_model()

    def load_model(self):
        """Charge le modèle sauvegardé."""
        print(f"Chargement du modèle depuis {self.model_path}...")
        try:
            self.model = pickle.load(open(self.model_path, 'rb'))
            print("Modèle chargé avec succès.")
        except FileNotFoundError:
            print(f"Erreur : le fichier du modèle '{self.model_path}' est introuvable.")
            print("Veuillez vous assurer que le fichier .pkl du modèle existe et que le chemin est correct.")
            self.model = None  # Le modèle ne peut pas être chargé
        except Exception as e:
            print(f"Erreur lors du chargement du modèle : {e}")
            self.model = None

    def preprocess_new_data(self, data):
        """
        Prétraite les nouvelles données brutes avant la prédiction.
        Cette fonction encapsule toutes les étapes de prétraitement
        et d'ingénierie des caractéristiques nécessaires.
        """
        print("Prétraitement des nouvelles données pour la prédiction...")
        new_data_processed = self.preprocessing_handler.create_new_features(data.copy())

        if self.preprocessing_handler.final_cols is not None:
            print("Alignement des colonnes des données traitées avec les colonnes finales...")
            final_columns = self.preprocessing_handler.final_cols
            missing_cols_in_test = set(final_columns) - set(new_data_processed.columns)
            for col in missing_cols_in_test:
                new_data_processed[col] = 0  # Ajouter les colonnes manquantes avec des zéros

            extra_cols_in_test = set(new_data_processed.columns) - set(final_columns)
            new_data_processed = new_data_processed.drop(columns=list(extra_cols_in_test), errors='ignore')

            new_data_processed = new_data_processed[final_columns]
            print("Alignement des colonnes terminé.")
        else:
            print("Attention : final_cols_for_align non chargé par PreprocessingHandler. Impossible d’aligner correctement les colonnes.")

        gc.collect()
        print("Nouvelles données prétraitées.")
        return new_data_processed

    def predict(self, data_for_prediction):
        """
        Effectue des prédictions sur les données préparées.
        """
        if self.model is None:
            print("Le modèle n’a pas pu être chargé. Impossible d’effectuer des prédictions.")
            return None, None

        print("Réalisation des prédictions...")
        try:
            if hasattr(self.model, 'predict_proba'):
                probabilities = self.model.predict_proba(data_for_prediction)[:, 1]  # Probabilité pour la classe positive
            elif hasattr(self.model, 'predict'):
                probabilities = self.model.predict(data_for_prediction)
            else:
                print("Le modèle ne possède pas de méthodes predict_proba ou predict.")
                return None, None

            if hasattr(self.model, 'predict_proba'):
                predictions_class = (probabilities > self.threshold).astype(int)
            elif hasattr(self.model, 'predict'):
                predictions_class = None  # Pas de prédiction de classe si le modèle ne donne pas de probas
                print("Le modèle ne fournit pas directement de probabilités de classe. Prédiction de classe ignorée.")
            else:
                predictions_class = None

            print("Prédictions terminées.")
            return probabilities, predictions_class
        except Exception as e:
            print(f"Erreur pendant la prédiction : {e}")
            return None, None

    def final_function_2(self, test_datapoint, targets_func_2=None):
        """
        Fonction finale pour la pipeline de prédiction. Orchestration du prétraitement, de la prédiction et de l'évaluation.

        Entrées :
            test_datapoint : pandas.DataFrame, les données brutes sur lesquelles effectuer des prédictions.
            targets_func_2 : pandas.Series ou numpy.array, facultatif, les vraies étiquettes pour l’évaluation.

        Sorties :
            Un dictionnaire contenant les résultats de la prédiction et les métriques d’évaluation si les cibles sont fournies.
        """
        start_time = time.time()
        print("\n----------------------------------------------------------------------------------------------------")
        print("Démarrage de la pipeline de prédiction...")

        # Étape 1 : Prétraiter les données d'entrée
        processed_data = self.preprocess_new_data(test_datapoint)

        # Étape 2 : Effectuer les prédictions
        # Passer les données transformées à la fonction predict
        probabilities, predictions = self.predict(processed_data)

        results = {
            "probabilities": probabilities,
            "predictions_class": predictions
        }

        # Étape 3 : Évaluer la performance du modèle si les étiquettes sont connues
        if targets_func_2 is not None and probabilities is not None:
            print("\nCalcul des métriques de performance...")
            try:
                roc_auc = roc_auc_score(targets_func_2, probabilities)
                print(f"Score ROC-AUC = {roc_auc}")
                results["roc_auc_score"] = roc_auc
                recall = recall_score(targets_func_2, predictions)
                print(f"Score Recall = {recall}")
                results["recall_score"] = recall
            except ValueError as e:
                print(f"Impossible de calculer le score ROC-AUC : {e}. Assurez-vous qu’il y ait au moins deux valeurs cibles différentes.")
                print(f"Impossible de calculer le score Recall : {e}. Assurez-vous qu’il y ait au moins deux valeurs cibles différentes.")
        else:
            print("Aucune étiquette fournie ou échec de la prédiction, évaluation ignorée.")

        end_time = time.time()
        time_elapsed = time.strftime("%H:%M:%S", time.gmtime(end_time - start_time))
        print(f"Temps total d’exécution de la prédiction = {time_elapsed}")
        print("----------------------------------------------------------------------------------------------------")

        return results


## Testing

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Scoring-Projet/data/application_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Scoring-Projet/data/application_test.csv')

In [ ]:
test_datapoint_func_1 = test_data.sample(1)
print("Out test query point for Testing Function 1 of pipeline is:")
display(test_datapoint_func_1)

Out test query point for Testing Function 1 of pipeline is:


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
11859,186139,Cash loans,F,N,N,0,157500.0,450000.0,21649.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Supposons que les DataFrames train_data et test_data sont déjà chargés
# Supposons que les classes PreprocessingHandler et FinalPipeline sont déjà définies

# Spécifiez le répertoire où se trouvent les fichiers nécessaires
file_directory = '/content/drive/MyDrive/Scoring-Projet/data/'  # Remplacez par votre chemin réel
# Spécifiez le chemin vers le modèle entraîné (utile ici pour le prétraitement,
# même si preprocess_new_data fait partie de FinalPipeline, qui charge le modèle automatiquement)
# On utilise le même chemin de modèle que dans la cellule f74a56bf pour rester cohérent.
model_path = '/content/drive/MyDrive/Scoring-Projet/data/Random_Forest_Model.pkl'


# Créez une instance de FinalPipeline (qui inclut PreprocessingHandler)
# Cela chargera tous les fichiers requis lors de l'initialisation
try:
    final_pipeline_instance_for_preprocessing_test = FinalPipeline(model_path=model_path, file_directory=file_directory)
    print("FinalPipeline instanciée avec succès.")

    # Appliquer le prétraitement et l’ingénierie des caractéristiques aux données de test
    test_datapoint_func_1 = test_data.sample(1)
    # Appeler preprocess_new_data sur l’instance
    test_data_processed = final_pipeline_instance_for_preprocessing_test.preprocess_new_data(test_datapoint_func_1.copy())
    print("Prétraitement des données de test terminé.")

    # Afficher les premières lignes des données traitées pour vérification
    print("\nAperçu des données de test traitées :")
    display(test_data_processed.head())

except FileNotFoundError as e:
    print(f"Erreur : Un fichier requis est introuvable. Veuillez vérifier le chemin file_directory et vous assurer que tous les fichiers nécessaires existent : {e}")
except Exception as e:
    print(f"Une erreur est survenue lors du prétraitement : {e}")

Chargement des fichiers requis pour le prétraitement...
Tous les fichiers requis ont été chargés avec succès.
Chargement du modèle depuis /content/drive/MyDrive/Scoring-Projet/data/Random_Forest_Model.pkl...
Modèle chargé avec succès.
FinalPipeline instanciée avec succès.
Prétraitement des nouvelles données pour la prédiction...
Creating new features...
Création de nouvelles variables.
Alignement des colonnes des données traitées avec les colonnes finales...
Alignement des colonnes terminé.
Nouvelles données prétraitées.
Prétraitement des données de test terminé.

Aperçu des données de test traitées :


,AMT_INTEREST_SUM_FIRST_2,STATUS_LAST_YEAR_1_MEAN_OVERALL,CODE_REJECT_REASON_MEAN_ALL,AMT_PAYMENT_TOTAL_CURRENT_MIN,CREDIT_TYPE_Cash loan (non-earmarked)_MEAN_OVERALL,AMT_CREDIT_SUM_ALL,ANNUITY_INCOME_RATIO_MAX_AGG_CODE_GENDER_NAME_FAMILY_STATUS_NAME_INCOME_TYPE,NONLIVINGAREA_MODE,CREDIT_TYPE_Car loan_MEAN_OVERALL,INTEREST_SHARE_MEAN_FIRST_2,...,NAME_TYPE_SUITE_MEAN_LAST_5,AMT_DOWN_PAYMENT_MEAN_ALL,CNT_INSTALMENT_FUTURE_MEAN_ACTIVE_SUM,AMT_DECLINED_SUM_ALL,CNT_INSTALMENT_FUTURE_MEAN_YEAR_1_SUM,CNT_PROLONGED_MAX_OVERDUE_MUL_MEAN_OVERALL,MAX_AMT_OVERDUE_DURATION_RATIO_MEAN_OVERALL,AMT_INCOME_TOTAL_MIN_AGG_FLAG_OWN_CAR_FLAG_OWN_REALTY_NAME_INCOME_TYPE,EXT_SOURCE_1_MEAN_AGG_NAME_EDUCATION_TYPE_NAME_INCOME_TYPE_OCCUPATION_TYPE,AMT_CREDIT_GOODS_RATIO_MIN_LAST_5
0,58402.980469,2.5,1.0,NaN,0.0,137691.0,0,NaN,0.0,0.424072,...,2.0,NaN,7.5,-25191.0,6.816406,NaN,NaN,0,0,1.22392


In [ ]:
# Assuming test_data DataFrame is already loaded
# Assuming the FinalPipeline class definition (from cell xkcFDMH9plFi) is already executed

# Spécifiez le chemin vers le modèle XGBoost entraîné
model_path = '/content/drive/MyDrive/Scoring-Projet/data/Random_Forest_Model.pkl' # Supposé être le modèle à utiliser
# Spécifiez le répertoire contenant les autres fichiers nécessaires (comme les fichiers .pkl pour le prétraitement)
file_directory = '/content/drive/MyDrive/Scoring-Projet/data/'

# Créez une instance de la classe FinalPipeline
try:
    final_pipeline_instance = FinalPipeline(model_path=model_path, file_directory=file_directory)
    print("FinalPipeline instanciée avec succès.")

    # Appliquez la méthode final_function_2 aux données de test brutes
    # La fonction gère le prétraitement en interne
    print("\nApplication de final_function_2 aux données de test...")
    prediction_results = final_pipeline_instance.final_function_2(test_datapoint_func_1) # Données de test brutes

    # Affichez les prédictions (probabilités et étiquettes de classe)
    print("\nRésultats de la prédiction :")
    print("Probabilités :", prediction_results.get("probabilities"))
    print("Étiquettes de classe prédites :", prediction_results.get("predictions_class"))

except FileNotFoundError as e:
    print(f"Erreur : Un fichier requis est introuvable. Veuillez vérifier les chemins du modèle et du répertoire de données : {e}")
except Exception as e:
    print(f"Une erreur est survenue pendant l'exécution du pipeline de prédiction : {e}")

Chargement des fichiers requis pour le prétraitement...
Tous les fichiers requis ont été chargés avec succès.
Chargement du modèle depuis /content/drive/MyDrive/Scoring-Projet/data/Random_Forest_Model.pkl...
Modèle chargé avec succès.
FinalPipeline instanciée avec succès.

Application de final_function_2 aux données de test...

----------------------------------------------------------------------------------------------------
Démarrage de la pipeline de prédiction...
Prétraitement des nouvelles données pour la prédiction...
Creating new features...
Création de nouvelles variables.
Alignement des colonnes des données traitées avec les colonnes finales...
Alignement des colonnes terminé.
Nouvelles données prétraitées.
Réalisation des prédictions...
Prédictions terminées.
Aucune étiquette fournie ou échec de la prédiction, évaluation ignorée.
Temps total d’exécution de la prédiction = 00:00:03
-----------------------------------------------------------------------------------------------

In [ ]:
# Sélectionner aléatoirement 50 lignes depuis les données d'entraînement comme jeu de test
test_datapoint_func_2 = train_data.sample(50).copy()

# Extraire la colonne cible 'TARGET' pour l'évaluation
targets_func_2 = test_datapoint_func_2.pop('TARGET')

# Afficher quelques exemples du jeu de test sélectionné
print("Quelques exemples de points de test pour tester la fonction 2 du pipeline :")
display(test_datapoint_func_2.head(5))

# Afficher les étiquettes cibles (valeurs réelles) correspondantes à ces points de test
print("Étiquettes cibles de ces points de données :")
print(targets_func_2.values[:5])

Quelques exemples de points de test pour tester la fonction 2 du pipeline :


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
241332,379433,Revolving loans,F,N,Y,1,90000.0,202500.0,10125.0,202500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
74762,186701,Cash loans,F,N,Y,1,337500.0,1358883.0,48937.5,1098000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
92762,207704,Cash loans,F,N,N,0,135000.0,352044.0,12645.0,247500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
73570,185315,Cash loans,F,N,Y,0,89100.0,314100.0,13833.0,225000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
60303,169919,Cash loans,M,Y,Y,0,202500.0,127350.0,8640.0,112500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


Étiquettes cibles de ces points de données :
[0 0 0 0 0]


In [ ]:
# Supposons que le DataFrame test_data est déjà chargé
# Supposons que la classe FinalPipeline (de la cellule xkcFDMH9plFi) est déjà définie

# Spécifiez le chemin vers le modèle XGBoost entraîné (ici un modèle Random Forest est utilisé)
model_path = '/content/drive/MyDrive/Scoring-Projet/data/Random_Forest_Model.pkl'  # On suppose que c’est le modèle à utiliser
# Spécifiez le répertoire contenant les autres fichiers nécessaires (comme les fichiers .pkl pour le prétraitement)
file_directory = '/content/drive/MyDrive/Scoring-Projet/data/'

# Créez une instance de la classe FinalPipeline
try:
    final_pipeline_instance = FinalPipeline(model_path=model_path, file_directory=file_directory)
    print("FinalPipeline instanciée avec succès.")

    # Appliquez la fonction final_function_2 aux données brutes de test
    # La fonction s’occupera du prétraitement en interne
    print("\nApplication de final_function_2 aux données de test...")
    prediction_results = final_pipeline_instance.final_function_2(test_datapoint_func_2, targets_func_2=targets_func_2)  # On passe les données brutes

    # Affiche les prédictions (probabilités et classes prédites)
    print("\nRésultats de la prédiction :")
    print("Probabilités :", prediction_results.get("probabilities"))
    print("Classes prédites :", prediction_results.get("predictions_class"))

except FileNotFoundError as e:
    print(f"Erreur : Un fichier requis est introuvable. Veuillez vérifier les chemins vers le modèle et le répertoire de données : {e}")
except Exception as e:
    print(f"Une erreur est survenue pendant le pipeline de prédiction : {e}")

Chargement des fichiers requis pour le prétraitement...
Tous les fichiers requis ont été chargés avec succès.
Chargement du modèle depuis /content/drive/MyDrive/Scoring-Projet/data/Random_Forest_Model.pkl...
Modèle chargé avec succès.
FinalPipeline instanciée avec succès.

Application de final_function_2 aux données de test...

----------------------------------------------------------------------------------------------------
Démarrage de la pipeline de prédiction...
Prétraitement des nouvelles données pour la prédiction...
Creating new features...
Création de nouvelles variables.
Alignement des colonnes des données traitées avec les colonnes finales...
Alignement des colonnes terminé.
Nouvelles données prétraitées.
Réalisation des prédictions...
Prédictions terminées.

Calcul des métriques de performance...
Score ROC-AUC = 0.5422222222222223
Score Recall = 0.4
Temps total d’exécution de la prédiction = 00:00:02
-------------------------------------------------------------------------

In [13]:
!git config --global user.email "datasansmythe@gmail.com"

In [14]:
!git config --global user.name "Datasansmythe"

In [15]:
!git clone https://github.com/datasansmythe/home-crdit.git

Cloning into 'home-crdit'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 24.53 KiB | 4.09 MiB/s, done.


In [ ]:
!cp -r /content/drive/MyDrive/Scoring-Projet /content/ton-depot/